# MONAI Label Deployment with MAIA Segmentation Portal

In [ ]:
%%writefile /home/maia-user/shared/MONAILabel/Spleen/spleen-segmentation_values.yaml

imagePullSecrets:
- "name": NONAME

image:
  repository: maiacloud/monet-x64-workstation-dgpu-linux-amd64
  pullPolicy: 
  # Overrides the image tag whose default is the chart appVersion.
  tag: "1.2-label"

volumes:
  #- name: holoscan-input
  #  persistentVolumeClaim:
  #    claimName: shared
  - name: model
    persistentVolumeClaim:
      claimName: shared
  - name: shm-size
    emptyDir:
      medium: Memory
command:
- 'bash'
- '-c'
- 'pip install monet-bundle && BUILD_OHIF=true monailabel start_server --app /opt/holoscan/monailabel/monaibundle --studies https://aida-workshop.maia-small.cloud.cbh.kth.se/orthanc-Zocu4r8gPAimKpAR/dicom-web/ --conf models MONetBundle'

ingress:
  enabled: true
  className: "nginx"
  annotations:
    cert-manager.io/cluster-issuer: cluster-issuer
    nginx.ingress.kubernetes.io/proxy-body-size: 8g
    nginx.ingress.kubernetes.io/proxy-read-timeout: "300"
    nginx.ingress.kubernetes.io/proxy-send-timeout: "300"
  hosts:
    - host: spleen-segmentation.aida-workshop.maia-small.cloud.cbh.kth.se
      paths:
        - path: /
          pathType: ImplementationSpecific
  tls:
    - secretName: spleen-segmentation-tls
      hosts:
        - spleen-segmentation.aida-workshop.maia-small.cloud.cbh.kth.se

volumeMounts:
  #- name: holoscan-input
  #  mountPath: /var/holoscan/input
  #  subPath: Task09_Spleen/imagesTs
  - name: model
    mountPath: /opt/holoscan/monailabel/monaibundle/model/MONetBundle
    subPath: MONAILabel/Spleen/MONetBundle
  - name: shm-size
    mountPath: /dev/shm

nodeSelector:
  kubernetes.io/hostname: maia-small-5

env:
  - name: MONAI_LABEL_DICOMWEB_SEARCH_FILTER
    value: '{"Modality": "CT", "PatientID": "spleen_*"}'
  - name: METADATA_JSON
    value: '{"description": "nnUNet Spleen Segmentation", "version": "0.0.1", "network_data_format": { "inputs": { "image": { "type": "image", "format": "hounsfield", "modality": "CT", "num_channels": 1, "spatial_shape": ["*", "*", "*"], "dtype": "float32", "value_range": [-1024, 1024], "is_patch_data": false, "channel_def": {"0": "image"} } }, "outputs": { "pred": { "type": "image", "format": "segmentation", "num_channels": 1, "spatial_shape": ["*", "*", "*"], "dtype": "float32", "value_range": [0, 1], "is_patch_data": false, "channel_def": {"0": "background", "1": "spleen"} } } } }'
#  - name: MONAI_LABEL_DICOMWEB_MODALITIES
#    value: '{"Order":["T1C","T1N","T2F","T2W"],"Reference":"T1C","Modalities":{"T1C":{"Modality":"MR","SeriesDescription":"T1C"},"T2F":{"Modality":"MR","SeriesDescription":"T2F"},"T2W":{"Modality":"MR","SeriesDescription":"T2W"},"T1N":{"Modality":"MR","SeriesDescription":"T1N"}}}'
#- name: MONAI_LABEL_DATASTORE
#  value: xnat
#- name: MONAI_LABEL_DATASTORE_USERNAME
#  value: admin
#- name: MONAI_LABEL_DATASTORE_PASSWORD
#  value: admin
#- name: MONAI_LABEL_DATASTORE_PROJECT
#  value: aida-demo
#- name: MONAI_LABEL_DATASTORE_ASSET_PATH
#  value: $XNAT_DATA_ARCHIVE
  #- name: MONAI_LABEL_AUTH_ENABLE
  #  value: "True"
  #- name: MONAI_LABEL_AUTH_REALM_URI
  #  value: https://iam.cloud.cbh.kth.se/realms/cloud
  #- name: MONAI_LABEL_AUTH_CLIENT_ID
  #  value: maia

In [ ]:
%%bash

cd $HOME
curl -fsSL -o get_helm.sh https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3
chmod 700 get_helm.sh
./get_helm.sh

In [ ]:
import subprocess

subprocess.run([
    "helm", "upgrade", "--install", "spleen-segmentation",
    "/home/maia-user/MAIA-AIDA-TechDays-Workshop/maia-monailabel-1.0.0.gz",
    "--namespace", "aida-workshop", "--values", "/home/maia-user/shared/MONAILabel/Spleen/spleen-segmentation_values.yaml"
],  capture_output=True,
    text=True)